In [94]:
import numpy as np
# import scipy 
# from scipy.optimize import brentq    #equation solver
from scipy import optimize
# from scipy.stats import norm
# from scipy.optimize import minimize
import matplotlib.pyplot as plt
# import numpy.polynomial.chebyshev as chebyshev
import numba
from scipy import interpolate
from itertools import islice 

# Aiyagari Model with Aggregate Uncertainty

$$
\max_{\left\{c_t, a_{t+1}\right\}_{t=0}^\infty} \mathbb{E}_t \left[\sum_{t=0}^{\infty} \beta^t \ln c_t\right]
$$
subject to: 

$$
 c_t + k_{t+1}\leq r_t k_t + w l_t + (1 - \delta) k_t ,\ \forall t
$$
 where $ \mathbb{L} = \{0,1\}$
 
and the natural borrowing limit
$$
a_{t}\geq - b,\ \forall t
$$
Note that 
$$\min_{\mathbb{L}} l_{t} = 0 ,\ \forall t$$

so that the natural debt limit is 0 for all consumers. 

The Bellman equation for this problem is:

$$
V(l,a) = \max_{c,a'} \left[ \frac{c^{1- \sigma}}{1-\sigma} + \beta \sum_{l' \in \mathbb{L}}  V(l',a') \Pi(l' | l) \right]
$$ 
subject to:
$$
 c + a' = (1+r) a + w l
$$
$$
a'\geq - b
$$

In [16]:
β  = 0.99
α  = 0.36
δ  = 0.025

# Aggregate and idiosyncratic shocks
Z  = np.array([0.99, 1.01])
π  = np.array([[7/8, 1/8], [1/8, 7/8]])
L  = np.array([0, 1])
Π  = np.array([[0.8507, 0.1229, 7/12, 3/32],
              [0.1159, 0.8361, 1/32, 7/20],
              [0.0243, 0.0021, 7/24, 1/32],
              [0.0091, 0.0389, 3/32, 21/40]])


#Unemployment levels conditional on aggregate state
U = np.array([.1, .04])

lstar = 1 - U
kstar = ( (1/β - 1+δ) /(α*(lstar)**(1-α))) ** (1 / (α - 1))
print(kstar)

@numba.njit
def labor(z):
    if z == 1.01:
        return .04
    else:
        return .1

@numba.njit
def f(z, K):
    L = labor(z)
    return z * K**α * L**(1 - α)

@numba.njit
def mpk(z, K):
    L = labor(z)
    return α * z *(L/K)**(1-α) - δ

@numba.njit
def util(c):
        return np.log(max(c, 1E-10))
    
# @numba.njit
# def K_D(r, z):
#     return (α/(r + δ))**(1/(1-α)) * lstar

@numba.njit
def wage(K, z):
    L = labor(z)
    return (1 - α) * z * (K/L) ** α

@numba.njit
def K_LOM(alpha, beta, K):
    return alpha + beta * np.log(K)


[34.19032818 36.4696834 ]


In [ ]:
#Initial guess law of motion for capital
alpha_guess     = np.array([0, 0])
beta_guess      = np.array([1, 1])



## Solving step 2: Finding the policy functions

In [19]:
def build_grid(Nk, NK, klbar, kubar, Klbar, Kubar):
    K = np.linspace(Klbar, Kubar, NK)
    k = klbar + np.divide(np.linspace(0, 0.5, Nk),0.5)**7 * (kubar - klbar)
    return K, k

K, k = build_grid(101, 8, 0, 100, 10, 80)

In [29]:
kplus = np.empty((len(k), len(K), len(L), len(Z)))
kplus += 0.9*k[:, np.newaxis, np.newaxis, np.newaxis]
print(kplus.shape)
print(kplus[80,1,0,0], kplus[80,7,1,1])

(101, 8, 2, 2)
37.748736000000015 37.748736000000015


### Deriving the Euler equation and household budget constraint

The euler equation is given by:

$$
u'(c(k,K,s,z)) = \beta \mathbb{E}_{s,z} \left(f_k(K_+, L_+, z_+) \ u'\left[c(k_+(k, K, s, z),K_+,s_+,z_+) \right] \right)
$$

and the budget constraint is:

$$
 c + k_+\leq r(K,L,z) k + w(K,L,z) l(s) + (1 - \delta) k ,\ \forall t
$$

In [117]:
#Fit bivariate spline over all states. 
def multispline(k, K, Y):

    tck = []
    for il, li in enumerate(L):
        tckiz = []
        for iz, zi in enumerate(Z):
            tckiz.append(interpolate.RectBivariateSpline(k, K, Y[:,:,il,iz]))
        tck.append(tckiz)

    return tck

#Evaluate the spline over all states
def multispline_eval(k, K, tck):

    output = np.zeros((len(k), len(K), len(L), len(Z)))
    for il, li in enumerate(L):
        for iz, zi in enumerate(Z):
            output[:,:,il,iz] =  tck[il][iz](k, K)

    return output

kplus = np.zeros((len(k), len(K), len(L), len(Z))) + 0.9*k[:, np.newaxis, np.newaxis, np.newaxis]
tck = multispline(k, K, kplus)
test = multispline_eval(k, K, tck)


[[[[-5.50020436e-20 -5.50020436e-20]
   [-5.50020436e-20 -5.50020436e-20]]

  [[ 8.08150411e-20  8.08150411e-20]
   [ 8.08150411e-20  8.08150411e-20]]

  [[ 1.15438487e-19  1.15438487e-19]
   [ 1.15438487e-19  1.15438487e-19]]

  ...

  [[ 7.48092252e-20  7.48092252e-20]
   [ 7.48092252e-20  7.48092252e-20]]

  [[ 2.77539430e-20  2.77539430e-20]
   [ 2.77539430e-20  2.77539430e-20]]

  [[-4.89613685e-21 -4.89613685e-21]
   [-4.89613685e-21 -4.89613685e-21]]]


 [[[ 5.63115619e-20  5.63115619e-20]
   [ 5.63115619e-20  5.63115619e-20]]

  [[-8.27391285e-20 -8.27391285e-20]
   [-8.27391285e-20 -8.27391285e-20]]

  [[-1.18186909e-19 -1.18186909e-19]
   [-1.18186909e-19 -1.18186909e-19]]

  ...

  [[-7.65903231e-20 -7.65903231e-20]
   [-7.65903231e-20 -7.65903231e-20]]

  [[-2.84147236e-20 -2.84147236e-20]
   [-2.84147236e-20 -2.84147236e-20]]

  [[ 5.01270684e-21  5.01270684e-21]
   [ 5.01270684e-21  5.01270684e-21]]]


 [[[-1.47669834e-21 -1.47669834e-21]
   [-1.47669834e-21 -1.47669834e-

In [70]:
def BC(kplus, kpplus):
    #Get consumption from budget constraint
    cons = f(kplus) - kpplus
    if cons < 0:
        return 10E-10
    else:
        return cons

@numba.njit
    def Euler(cplus, kplus):
    # Get consumption from Euler
    euler = cplus / (beta * fk(kplus))
    return euler

@numba.njit
def backward_iterate_II(kplus, k, K, L, Z):
    # init
    kpplus = np.empty_like(kplus)
    cendog = np.empty(kplus)
    kendog = np.empty(kplus)
    cplus = np.empty(kplus)
    
    # Use policy function to get K''
    
    spline = multispline(k, K kplus)
    kpplus = spline(kplus, K)
#     print(kpplus)

    # Use BC tomorrow, Euler today and BC today to get back policy func
    for ik, k_cur in enumerate(k):
        
        cplus[ik]  = BC(kplus[ik], kpplus[ik])    #Budget constraint tomorrow
        cendog[ik] = Euler(cplus[ik], kplus[ik])  # Euler equation today
        kendog[ik] = min(max(f(k[ik]) - cendog[ik], 0), kbar) # Budget constraint today lower bar = 0, upper bar to define
        
    return kendog


IndentationError: unexpected indent (<ipython-input-70-055f7bfe4353>, line 10)

In [ ]:
def ss_policy_II(k, K, L, Z):
    
    kplus = np.zeros((len(k), len(K), len(L), len(Z))) + 0.9*k[:, np.newaxis, np.newaxis, np.newaxis]
    knew = np.empty_like(kplus)
    
    for it in range(1000):
#         print("Iteration number:", it)
        knew = backward_iterate_II(kplus, k, K, L, Z) 
        
        if it % 10 == 1 and  np.linalg.norm(knew - kplus) < 1E-10:
            print(f'convergence in {it} iterations!')
            tck_out = interpolate.splrep(k, knew)
            return tck_out
        
        kplus = np.copy(knew)
    

In [ ]:
%time tck = ss_policy_II(k, 5*kstar)

In [ ]:
savings = interpolate.splev(k, tck)
c = f(k) - savings
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.set_title('Consumption')
ax2.set_title('Capital accumulation')
ax1.plot(k, c)
ax2.plot(k, savings)
ax1.legend()
plt.tight_layout()

In [17]:
test = np.empty((100, 100, 2, 2))

array([[[[6.94731427e-310, 6.94731427e-310],
         [4.63933242e-310, 4.63933242e-310]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        ...,

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]]],


       [[[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        ...,

        [[7.29111856e-304, 7.29111856e-304],
         [7.29111856e-304, 7.29111856e-304]],

        [[7.29111856e-304, 7.29111856e-304],
     

In [34]:
print(kplus.shape)

(101, 8, 2, 2)
